In [1]:
import torch
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
DATA_DIR = '../data'

# Download training data
training_data = datasets.FashionMNIST(
    root=DATA_DIR,
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data
test_data = datasets.FashionMNIST(
    root=DATA_DIR,
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64 

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# Get cpu or gpu device for training
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'Using {device} device')


# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
# Define the loss function and the optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


epochs = 5 
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(train_dataloader, model, loss_fn)
print('Done.')

Epoch 1
-------------------------------
loss: 2.304973  [   64/60000]
loss: 2.294636  [ 6464/60000]
loss: 2.274902  [12864/60000]
loss: 2.272508  [19264/60000]
loss: 2.263643  [25664/60000]
loss: 2.223203  [32064/60000]
loss: 2.240774  [38464/60000]
loss: 2.209035  [44864/60000]
loss: 2.202449  [51264/60000]
loss: 2.179712  [57664/60000]
Test Error: 
 Accuracy: 42.2%, Avg loss: 2.174075 

Epoch 2
-------------------------------
loss: 2.181840  [   64/60000]
loss: 2.177519  [ 6464/60000]
loss: 2.127230  [12864/60000]
loss: 2.145851  [19264/60000]
loss: 2.104316  [25664/60000]
loss: 2.039657  [32064/60000]
loss: 2.076097  [38464/60000]
loss: 2.005441  [44864/60000]
loss: 1.997681  [51264/60000]
loss: 1.951716  [57664/60000]
Test Error: 
 Accuracy: 59.7%, Avg loss: 1.943256 

Epoch 3
-------------------------------
loss: 1.963106  [   64/60000]
loss: 1.948655  [ 6464/60000]
loss: 1.843022  [12864/60000]
loss: 1.886464  [19264/60000]
loss: 1.783774  [25664/60000]
loss: 1.721122  [32064/600

In [8]:
# Save the model
MODEL_DIR = '../models'
MODEL_NAME = f'{MODEL_DIR}/1_quickstart.pth'
torch.save(model.state_dict(), MODEL_NAME)
print(f'Saved PyTorch model to {MODEL_NAME}')

Saved PyTorch model to ../models/1_quickstart.pth


In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load(MODEL_NAME))

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
